In [1]:
import mlflow
mlflow.set_tracking_uri("http://34.202.229.179:5000")


c:\Users\Deepu\anaconda3\envs\nlp\lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [5]:
df= pd.read_csv(r"C:\Users\Deepu\OneDrive\Desktop\Capstone1\reddit_preprocessing.csv").dropna()
df.shape

(36750, 7)

In [7]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000   # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get best parameters
    best_params = study.best_params

    # Train final best model
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )
    best_model.fit(X_train, y_train)

    # Predictions and classification report
    y_pred = best_model.predict(X_test)
    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    # Log metrics in MLflow
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Save model in MLflow and get the path
    model_info = mlflow.sklearn.log_model(best_model, artifact_path="theXGBoost_model")
    saved_model_path = model_info.model_uri


    print("Best parameters:", best_params)
    print(classification_report(y_test, y_pred))
    print("Best model saved at:", saved_model_path)

    return saved_model_path

# Run the experiment
best_model_path = run_optuna_experiment()


[I 2025-08-14 18:16:48,529] A new study created in memory with name: no-name-2ee071a9-b35a-4125-95c1-b0122d85a4bd
[I 2025-08-14 18:16:57,630] Trial 0 finished with value: 0.8933100824350032 and parameters: {'n_estimators': 123, 'learning_rate': 0.047244726107468896, 'max_depth': 9}. Best is trial 0 with value: 0.8933100824350032.
[I 2025-08-14 18:17:36,456] Trial 1 finished with value: 0.7821813570069752 and parameters: {'n_estimators': 241, 'learning_rate': 0.00034428322411388183, 'max_depth': 9}. Best is trial 0 with value: 0.8933100824350032.
[I 2025-08-14 18:17:39,186] Trial 2 finished with value: 0.7141724793912492 and parameters: {'n_estimators': 124, 'learning_rate': 0.0003316589725022364, 'max_depth': 3}. Best is trial 0 with value: 0.8933100824350032.
[I 2025-08-14 18:17:49,072] Trial 3 finished with value: 0.8749207355738744 and parameters: {'n_estimators': 126, 'learning_rate': 0.03136970433959323, 'max_depth': 8}. Best is trial 0 with value: 0.8933100824350032.
[I 2025-08-1

MlflowException: API request to http://34.202.229.179:5000/api/2.0/mlflow/runs/create failed with timeout exception HTTPConnectionPool(host='34.202.229.179', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000016199B6CDF0>, 'Connection to 34.202.229.179 timed out. (connect timeout=120)')). To increase the timeout, set the environment variable MLFLOW_HTTP_REQUEST_TIMEOUT (default: 120, type: int) to a larger value.